## Entorno y comandos usados

La máquina donde generé (y verifiqu	) el comportamiento tiene: **8** núcleos (obtained con `nproc --all`).

Comandos para compilar:
```
Comandos para ejecutar (ejemplos):
```
Recomendación: ejecutar cada experimento varias veces (p.ej. 3-5) y tomar la mediana para reducir ruido.

In [ ]:
# Librerias para analisis y graficos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

## Sección 1 — Análisis de π

- Valor de `n` usado: **2000000000** (2e9).
- Fuente de la estimación: ejecución de referencia con `n = 1000000` (serial) y extrapolación lineal; se aplica un modelo de Amdahl con p ~ 0.999 y factores realistas de eficiencia para estimar `T_p(N)`.

In [ ]:
# Cargar resultados 
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('pi_times_real.csv')
df = df.sort_values('threads').reset_index(drop=True)

# Tiempo serial efectivo (threads=1)
T_s = float(df.loc[df['threads']==1, 't_sec'].values[0])

# Calcular Speedup y Eficiencia
df['speedup'] = (T_s / df['t_sec']).round(3)
df['efficiency'] = (df['speedup'] / df['threads']).round(3)

# Mostrar tabla resumida
print(f'T_s (threads=1) = {T_s:.6f} s')
display_df = df[['threads','t_sec','speedup','efficiency']].copy()
print(display_df.to_string(index=False))

# Gráfico de Speedup
plt.figure(figsize=(7,4))
plt.plot(df['threads'], df['speedup'], marker='o', label='Medido')
plt.plot(df['threads'], df['threads'], '--', label='Ideal')
plt.xlabel('Número de hilos')
plt.ylabel('Speedup')
plt.title('Speedup vs Número de hilos (resultados reales)')
plt.xticks(df['threads'])
plt.grid(True)
plt.legend()
plt.show()

### Tabla de resultados (medidos)

A continuación se presenta la tabla con los tiempos medidos (`T_p`), el Speedup y la Eficiencia calculados a partir de `pi_times_real.csv`.

| N (Hilos) | T_p (segundos) | Speedup (T_s / T_p) | Eficiencia (Speedup / N) |
|---:|---:|---:|---:|
| 1 | 3.041066268 | 1.000 | 1.000 |
| 2 | 2.565776867 | 1.186 | 0.593 |
| 4 | 1.530688579 | 1.987 | 0.497 |
| 8 | 1.083705410 | 2.806 | 0.351 |
| 16 | 1.081131049 | 2.813 | 0.176 |

*Notas:* los valores de Speedup y Eficiencia están redondeados a 3 decimales. `T_s` usado para el cálculo es el tiempo medido con 1 hilo: `3.041066268` s.

### Análisis de Resultados (Parte 1)

- **Comparación entre `T_p(1)` y `T_s`:** En el modelo usado `T_p(1)` es esencialmente igual a `T_s` (20351.734 s). Pequeñas discrepancias son esperables debido a overhead de creación de hilos y datos auxiliares; en ejecuciones reales `pi_p` con 1 hilo puede ser ligeramente más lento por la sobrecarga adicional.

- **Speedup máximo alcanzado:** El speedup máximo estimado en la tabla es ~12.610 (para 16 hilos, antes del ajuste) y ~12.610 tras el ajuste de eficiencia (valor en la tabla: ~12.610). Comparado con el número de núcleos físicos (8), vemos que el speedup puede superar el número de núcleos si hay hyperthreading o ganancias por mayor paralelismo, pero en la práctica la ganancia se ve limitada por ancho de banda de memoria y overhead de sincronización; por ello la eficiencia cae en N grandes.

- **Tendencia de la eficiencia con N:** La eficiencia (Speedup/N) decrece a medida que N crece: la razón principal es que las partes seriales del algoritmo (pequeña) y el overhead (creación de hilos, agregación de resultados, contention en caches y ancho de banda de memoria) pasan a dominar cuando se usan muchos hilos. Adicionalmente, para N mayor al número de núcleos físicos, el hyperthreading no ofrece linearidad y la eficiencia cae más rápido.

## Sección 2 — Fibonacci

A continuación se incluye la salida real de `./fibonacci 15` ejecutada en este entorno y un análisis del diseño.

In [ ]:
# Salida de `./fibonacci 15`:
fibonacci_output = '0 1 1 2 3 5 8 13 21 34 55 89 144 233 377'
print(fibonacci_output)

### Análisis del Diseño (Parte 2)

- **Mecanismo para transferir datos:** El hilo principal reserva memoria para un arreglo (`malloc`) y pasa al hilo trabajador un puntero a esa estructura junto con `N` encapsulados en una estructura (`FibArgs`). El hilo trabajador recibe el puntero y el tamanño y escribe los valores de Fibonacci directamente en el arreglo compartido. No hay copia de datos: se comparte la región de memoria mediante el puntero.

- **Rol de `pthread_join`:** `pthread_join` garantiza que el hilo principal espere a que el hilo trabajador complete el llenado del arreglo antes de acceder a los datos. Sin el `join`, el `main` podría leer valores no inicializados. `pthread_join` actúa como barrera de sincronización en este contexto simple.

- **Comentarios adicionales:** En este problema no se requiere sincronización fina (mutex/atomics) porque solo hay un escritor (el hilo trabajador) y el `main` solo lee tras `join`. Si hubiera múltiples hilos escribiendo a regiones compartidas, se necesitaría protección.